<a href="https://colab.research.google.com/github/AvaneSauhard/HumanActivityRecognition/blob/main/HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
from google.colab import drive
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.utils import to_categorical
from sklearn.utils import resample
import itertools
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def load_file(filepath):
  data=pd.read_csv(filepath,header=None,delim_whitespace=True)
  return data.values

In [ ]:
def load_group(filenames,filepaths):
  load=list()
  for name in filenames:
    data=load_file(filepaths+name)
    load.append(data)
  load=np.dstack(load)
  return load


In [ ]:
filepath_train="drive/MyDrive/UCI HAR Dataset/train/Inertial Signals/"
filename_train=list()
filename_train+=['total_acc_x_train.txt','total_acc_y_train.txt','total_acc_z_train.txt']
filename_train+=['body_acc_x_train.txt','body_acc_y_train.txt','body_acc_z_train.txt']
filename_train+=['body_gyro_x_train.txt','body_gyro_y_train.txt','body_gyro_z_train.txt']
train_X=load_group(filename_train,filepath_train)
train_Y=load_file("drive/MyDrive/UCI HAR Dataset/train/y_train.txt")

In [ ]:
print(train_X.shape,train_Y.shape)

(7352, 128, 9) (7352, 1)


In [ ]:
filepath_test="drive/MyDrive/UCI HAR Dataset/test/Inertial Signals/"
filename_test=list()
filename_test+=['total_acc_x_test.txt','total_acc_y_test.txt','total_acc_z_test.txt']
filename_test+=['body_acc_x_test.txt','body_acc_y_test.txt','body_acc_z_test.txt']
filename_test+=['body_gyro_x_test.txt','body_gyro_y_test.txt','body_gyro_z_test.txt']
test_X=load_group(filename_test,filepath_test)
test_Y=load_file("drive/MyDrive/UCI HAR Dataset/test/y_test.txt")

In [ ]:
print(test_X.shape,test_Y.shape)

(2947, 128, 9) (2947, 1)


In [ ]:
train_Y=train_Y-1
test_Y=test_Y-1

In [ ]:
train_Y=to_categorical(train_Y)
test_Y=to_categorical(test_Y)

In [ ]:
print(train_X.shape,train_Y.shape,test_X.shape,test_Y.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def scale_data(standardize):
  cut= int(train_X.shape[1]/2)
  longX=train_X[:,-cut:,:]
  longX=longX.reshape((longX.shape[0]*longX.shape[1],longX.shape[2]))
  std_train_X= train_X.reshape((train_X.shape[0]*train_X.shape[1],train_X.shape[2]))
  std_test_X= test_X.reshape((test_X.shape[0]*test_X.shape[1],test_X.shape[2]))
  if standardize:
    s = StandardScaler()
    s.fit(longX)
    longX = s.transform(longX)
    std_train_X = s.transform(std_train_X)
    std_test_X = s.transform(std_test_X)
  std_train_X = std_train_X.reshape((train_X.shape))
  std_test_X = std_test_X.reshape((test_X.shape))
  return std_train_X,std_test_X





In [ ]:
def build_model(params):
  #train_X,test_X= scale_data(params)
  timestamps, features, output=train_X.shape[1],train_X.shape[2],train_Y.shape[1]
  #head1
  inputs1= input(shape=(timestamps,features))
  conv1 = Conv1D(filters=64,kernel_size=3,activation='relu')(inputs1)
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  #head2
  inputs2= input(shape=(timestamps,features))
  conv2 = Conv1D(filters=64,kernel_size=5,activation='relu')(inputs2)
  drop2 = Dropout(0.5)(conv2)
  pool2 = MaxPooling1D(pool_size=2)(drop2)
  flat2 = Flatten()(pool2)
  #head3
  inputs3= input(shape=(timestamps,features))
  conv3 = Conv1D(filters=64,kernel_size=11,activation='relu')(inputs3)
  drop3 = Dropout(0.5)(conv3)
  pool3 = MaxPooling1D(pool_size=2)(drop3)
  flat3 = Flatten()(pool3)
  merged = concatenate([flat1,flat2,flat3])
  dense = Dense(100,activation='relu')(merged)
  output = Dense(train_Y.shape[1],activation='softmax')(dense)

  model = Model(inputs=[inputs1,inputs2,inputs3],outputs=output)
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.fit([train_X,train_X,train_X],train_Y,epochs=8,batch_size=64)
  accuracy=model.evaluate([test_X,test_X,test_X],test_Y,batch_size=64)
  return accuracy

In [ ]:
n_params=[2,3,5,7,11]
for p in n_params:
  scores=list()
  score=build_model(p)
  score = score*100
  scores.append(score)
for i in range(len(scores)):
  print(scores)

TypeError: ignored